In [1]:
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ================= CONFIGURATION =================
# 1. Load your data (Result from previous step)
adata = sc.read_h5ad(
    "../../data/Kidney_ST/Kidney_CosMx_BANKSY_Calculated.h5ad")

# 2. Define the Target Interaction
# (Change this to the Domain ID that showed the highest recruitment in your plots)
TARGET_DOMAIN = '7'
TCM_THRESHOLD_PERCENTILE = 75  # Top 25% of interacting cells

# ================= PROCESSING =================

print("Classifying Recruiting vs Recruited cells...")

# 1. Create a container for the status
adata.obs['Recruitment_Status'] = 'Background'

# 2. Identify "Recruiting" Cells (The Niche)
# These are simply all cells belonging to the target domain
is_recruiter = adata.obs['BANKSY_Domain'].astype(str) == TARGET_DOMAIN
adata.obs.loc[is_recruiter,
              'Recruitment_Status'] = f'Recruiting Niche (Dom {TARGET_DOMAIN})'

# 3. Identify "Recruited" Cells (The Immune cells with High TCM)
# We assume 'TCM_Global' was calculated in the previous script.
# If TCM is NaN, it means the cell wasn't close enough or wasn't immune.

# Calculate the threshold value for "High Recruitment"
tcm_cutoff = np.nanpercentile(
    adata.obs['TCM_Global'], TCM_THRESHOLD_PERCENTILE)
print(
    f"TCM Score Cutoff (Top {100-TCM_THRESHOLD_PERCENTILE}%): {tcm_cutoff:.4f}")

is_immune = adata.obs['cellType_CosMx_2'] == 'Immune'
has_high_score = adata.obs['TCM_Global'] >= tcm_cutoff

# Label them
adata.obs.loc[is_immune & has_high_score,
              'Recruitment_Status'] = 'Highly Recruited Immune'
adata.obs.loc[is_immune & ~has_high_score,
              'Recruitment_Status'] = 'Bystander Immune'

# ================= PLOTTING =================

# Set specific colors: Red for Niche, Blue for Recruited, Grey for others
palette = {
    'Background': '#e0e0e0',  # Light Grey
    'Bystander Immune': '#add8e6',  # Light Blue
    'Highly Recruited Immune': '#00008b',  # Dark Blue
    f'Recruiting Niche (Dom {TARGET_DOMAIN})': '#d62728'  # Red
}

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# --- Plot A: UMAP ---
# Ensure UMAP is calculated
if 'X_umap' not in adata.obsm:
    print("Calculating UMAP...")
    sc.pp.neighbors(adata)
    sc.tl.umap(adata)

# We plot in layers to ensure the important cells are on top
sc.pl.umap(
    adata,
    color='Recruitment_Status',
    palette=palette,
    ax=axes[0],
    show=False,
    frameon=False,
    title=f"Recruitment on UMAP (Domain {TARGET_DOMAIN})",
    # Plotting order: Background first, then Bystanders, then Key actors
    groups=['Background', 'Bystander Immune', 'Highly Recruited Immune',
            f'Recruiting Niche (Dom {TARGET_DOMAIN})']
)

# --- Plot B: Spatial ---
# Use spatial_corrected or x_slide_mm/y_slide_mm
sc.pl.spatial(
    adata,
    color='Recruitment_Status',
    coords_name='spatial_corrected',  # Ensure this key exists from previous script
    palette=palette,
    ax=axes[1],
    show=False,
    frameon=False,
    title=f"Spatial Recruitment Map",
    spot_size=30,  # Adjust spot size for visibility
    na_color='#e0e0e0'
)

plt.tight_layout()
plt.savefig("Recruitment_UMAP_Spatial.png", dpi=300)
plt.show()

# ================= EXPORT LIST =================
# Save the IDs of the recruited cells for downstream analysis (e.g., DE analysis)
recruited_cells = adata[adata.obs['Recruitment_Status']
                        == 'Highly Recruited Immune']
recruited_cells.obs.to_csv("Highly_Recruited_Immune_Cells.csv")
print(
    f"Saved {recruited_cells.shape[0]} highly recruited immune cells to CSV.")


Classifying Recruiting vs Recruited cells...


KeyError: 'TCM_Global'